In [57]:

from dotenv import load_dotenv
from langchain_openai import  AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain,create_history_aware_retriever


import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoTokenizer, AutoModel
from langchain.embeddings.base import Embeddings

import os
import pandas as pd
import hashlib
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document



load_dotenv()

os.environ['OPENAI_API_TYPE']=os.getenv("AL_OPENAI_API_TYPE")
os.environ['OPENAI_API_VERSION']=os.getenv("AL_OPENAI_API_VERSION")
os.environ['AZURE_OPENAI_ENDPOINT']=os.getenv("AL_AZURE_OPENAI_ENDPOINT")
os.environ['OPENAI_API_KEY']=os.getenv("AL_OPENAI_API_KEY")
os.environ['DEPLOYMENT_NAME']=os.getenv("AL_DEPLOYMENT_NAME")

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("AL_LANGCHAIN_PROJECT")

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import  openai
openai.api_key =os.getenv("AL_OPENAI_API_KEY")
openai.api_type = os.getenv("AL_OPENAI_API_TYPE")
openai.api_version = os.getenv("AL_OPENAI_API_VERSION")
openai.deployment_name = os.getenv("AL_DEPLOYMENT_NAME")
openai.api_base = os.getenv("AL_OPENAI_ENDPOINT")



In [58]:
def get_embedding(text, model="text-embedding-ada-002"):
    response = openai.Embedding.create(
      input=text,
      model=model
    )
    return response['data'][0]['embedding'] 

In [59]:
import openai

def search_with_embeddings(query_embedding):
    response = openai.Search.create(
      documents=["doc1", "doc2", "doc3"],  # Example docs
      query_embedding=query_embedding,
      model="ada"
    )
    return response['data']


In [60]:
import openai

# Define a function to retrieve context
def retrieve_context(input_text):
    # Get embedding for the input query
    embedding = get_embedding(input_text)

    # Perform retrieval with embedding
    search_results = search_with_embeddings(embedding)

    # Combine context from search results
    context = "\n".join([doc["text"] for doc in search_results])
    return context


In [61]:
def contextualize_question(chat_history, latest_question):
    prompt = (
        "Given a chat history and the latest user question "
        "which might reference context in the chat history, "
        "formulate a standalone question which can be understood "
        "without the chat history. Do NOT answer the question, "
        "just reformulate it if needed."
    )
    reformulated_question = openai.completions.create(
        model="gpt-4",
        prompt=chat_history + [{"role": "user", "content": latest_question}]
    )
    return reformulated_question['choices'][0]['message']['content']


In [62]:
import openai

def question_answer_pipeline(question, chat_history):
    # Reformulate the question (if needed)
    standalone_question = contextualize_question(chat_history, question)
    
    # Retrieve context for the question
    context = retrieve_context(standalone_question)

    # Generate the answer based on the retrieved context
    response = openai.completions.create(
        model="gpt-4",  # Change to your desired model
        prompt=ChatPromptTemplate(
            question=standalone_question,
            context=context
        )
    )
    
    return response['choices'][0]['message']['content']



In [63]:
chat_store = {}

def get_chat_history(session_id):
    if session_id not in chat_store:
        chat_store[session_id] = []
    return chat_store[session_id]

def save_chat_history(session_id, messages):
    if session_id in chat_store:
        chat_store[session_id].extend(messages)
    else:
        chat_store[session_id] = messages


In [64]:
session_id = "session-1"
question = "What are the phases mentioned in the document?"

# Get chat history
chat_history = get_chat_history(session_id)

# Get the answer from the pipeline
answer = question_answer_pipeline(question, chat_history)

# Save the conversation
save_chat_history(session_id, [
    {"role": "user", "content": question},
    {"role": "assistant", "content": answer}
])

print(answer)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: de498b61********************d0fb. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}